In [1]:
# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

In [2]:
from scripts.load_data import *
from scripts.preprocess import *
from scripts.helper import *
from scripts.applicant_prediction import *
from scripts.transform_data import *
from scripts.predict_sarima import *

In [3]:
data_individual, data_cumulative, data_student_numbers, data_latest, data_distances, data_weighted_ensemble = load_data()

In [4]:
data_individual = read_and_preprocess(data_individual, data_distances, FilterStudentYear.HIGHER_YEARS)

In [5]:
predict_year = 2023
predict_week = 7

max_year = data_individual["Collegejaar"].max()
max_week = get_max_week(data_individual[data_individual["Collegejaar"] == predict_year]["Weeknummer"])

In [6]:
predicties = predict_preapplication(data_individual, predict_year, max_year, predict_week)

In [7]:
data_individual.loc[(data_individual["Collegejaar"] == predict_year) & (data_individual["Weeknummer"].isin(get_weeks_list(predict_week))), 'Inschrijvingen_predictie'] = predicties

In [8]:
data_individual = transform(data_individual, predict_year, max_week)

In [9]:
data_individual = transform_data(data_individual, targ_col='Cumulative_sum_within_year')

In [10]:
# dict = {'Collegejaar': [], 'Weeknummer': [], 'Faculteit': [], 'Examentype': [], 'Herkomst': [], 'Croho groepeernaam': [], 'SARIMA_individual': []}

# for index, row in data_individual[data_individual["Collegejaar"] == predict_year].iterrows():
#     sarima_ind = predict_with_sarima(data_individual, None, row["Croho groepeernaam"], row["Herkomst"], predict_week, predict_year, max_year)

#     dict['Collegejaar'].append(predict_year)
#     dict['Weeknummer'].append(predict_week)
#     dict['Faculteit'].append(row["Faculteit"])
#     dict['Examentype'].append(row["Examentype"])
#     dict['Herkomst'].append(row["Herkomst"])
#     dict['Croho groepeernaam'].append(row["Croho groepeernaam"])
#     dict['SARIMA_individual'].append(sarima_ind)

# dict = pd.DataFrame(dict)
# dict.to_excel("individual_output.xlsx", index=False)

In [11]:
data_to_predict = data_individual[data_individual["Collegejaar"] == predict_year]

nr_CPU_cores = os.cpu_count()
chunk_size = math.ceil(len(data_to_predict) / nr_CPU_cores) # Make as much chunks as you have CPU cores

chunks = [data_to_predict[i:i + chunk_size] for i in range(0, len(data_to_predict), chunk_size)]

In [12]:
def sarima_parallel(data_individual, row, predict_week, predict_year, max_year):
    print("Opleiding:", row["Croho groepeernaam"], " Herkomst:", row["Herkomst"])
    return predict_with_sarima(data_individual, None, row["Croho groepeernaam"], row["Herkomst"], predict_week, predict_year, max_year)

In [13]:
print("Start parallel work")

results = joblib.Parallel(n_jobs=nr_CPU_cores)(
        joblib.delayed(sarima_parallel)(data_individual, row, predict_week, predict_year, max_year) for chunk in chunks for _, row in chunk.iterrows()
    )

data_to_predict["SARIMA_individual"] = results
data_to_predict = data_to_predict[["Collegejaar", "Faculteit", "Herkomst", "Examentype", "Croho groepeernaam", "SARIMA_individual"]]
data_to_predict.to_excel("individual_output_higher-years.xlsx", index=False)

Start parallel work
